In [39]:
import csv
import json
import re
import numpy as np
import pandas as pd
from pyecharts.charts import Bar,Pie, Timeline
from pyecharts.commons.utils import JsCode

from pyecharts import options as opts
from collections import Counter, OrderedDict

In [40]:
df_2018 = pd.read_csv('/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/kaggle-survey-2018/multipleChoiceResponses.csv')
df_2018 = df_2018.drop([0])
df_2019 = pd.read_csv('/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/kaggle-survey-2019/multiple_choice_responses.csv')
df_2019 = df_2019.drop([0])

/home/harisyam/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/harisyam/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,3,7,19,34,47,49,50,51,52,53,54,68,81,94,96,109,115,130,139,147,154,167,180,193,206,219,232,245) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
df_responses = pd.read_csv('./kaggle-survey-2020/kaggle_survey_2020_responses.csv')
df_responses = df_responses.drop([0])
df_responses.head()

/home/harisyam/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
1,1838,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN
2,289287,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,860,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,507,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,Python,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,78,30-34,Man,Japan,Master’s degree,Software Engineer,3-5 years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
q1 = df_responses['Q1'].value_counts().sort_index()
q1=q1.reset_index()
q1.columns=['Age-Group', '# of responses']

bar=Bar().add_xaxis(q1['Age-Group'].values.tolist())\
.add_yaxis('num responses', q1['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Age Group", subtitle="2020 Kaggle Survey"),
             xaxis_opts=opts.AxisOpts(name='Age-Group'))\
.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="Maximum no. of responses"),
                opts.MarkPointItem(type_="min", name="Minimum no. of responses"),
            ]
        ),
    )

bar.render('fig1.html')

'/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/fig1.html'

In [43]:
df_responses['Q4'] = df_responses['Q4'].map({'Master’s degree': 'Master',
                                       'Bachelor’s degree': 'Bachelor', 'Doctoral degree': 'Higher than Master',
                                       'Some college/university study without earning a bachelor’s degree': 'Lower than Bachelor',
                                      'Professional degree': 'Higher than Master', 'No formal education past high school':'Lower than Bachelor'})

q4=df_responses['Q4'].value_counts().to_frame()
q4=q4.reset_index()
q4.columns=['Degree', '# of responses']

bar=Bar().add_xaxis(q4.Degree.values.tolist())\
.add_yaxis('', q4['# of responses'].values.tolist())\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Education Level",
                                           subtitle="2020 Kaggle Survey"), xaxis_opts=opts.AxisOpts(name='Degree'))

bar.render('fig2.html')

'/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/fig2.html'

In [44]:
df_responses['Q2'] = df_responses['Q2'].apply(lambda x : 'ETC' if x not in ['Man', 'Woman'] else x)
total_dist=df_responses.Q2.value_counts(normalize=True)
data_q1q2 = df_responses[df_responses['Q2'] != 'ETC'].groupby(['Q2'])['Q1'].value_counts().unstack().sort_index()
man = data_q1q2.loc['Man']
woman = data_q1q2.loc['Woman']

In [45]:
man=man.reset_index()
man.columns = ['Age-Group', '# of responses']

woman=woman.reset_index()
woman.columns = ['Age-Group', '# of responses']

In [46]:
total_dist = total_dist.reset_index()

In [47]:
total_dist.columns = ["Group", "Percent"]
total_dist.Percent = np.round((total_dist.Percent * 100),2)
total_dist['Group'] = total_dist['Group'].apply(lambda x : 'ETC' if x not in ['Man', 'Woman'] else x)

In [48]:
total_dist['Group'] = total_dist.Group.map({"Man":"Men","Woman":"Women", "ETC":"ETC"})

In [49]:
bar=Bar().add_xaxis(man['Age-Group'].values.tolist())\
.add_yaxis('Men', man['# of responses'].values.tolist(), stack="stack1")\
.add_yaxis('Women', woman['# of responses'].values.tolist(), stack="stack1")\
.set_series_opts(label_opts=opts.LabelOpts(position="inside"))\
.reversal_axis()\
.set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents based on Sex"),
                xaxis_opts=opts.AxisOpts(name='# responses'), yaxis_opts=opts.AxisOpts(name='Age-Group'))\

pie = (
    Pie()
    .add(
        series_name="percent responses",
        data_pair=[list(z) for z in zip(total_dist.Group, total_dist.Percent)],
        radius=["30%","40%"],
        center=["75%", "35%"],
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}%"),
        tooltip_opts=opts.TooltipOpts(formatter="{b}:{c}%"))
)

bar.overlap(pie)

bar.render('fig3.html')

'/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/fig3.html'

In [50]:
continents_df = pd.read_csv("./kaggle-survey-2020/2020 Continents.csv")
continents_dict = d = {k:v for k,v in zip(continents_df["Country"],continents_df["Continents"])}
df_responses["Continents"] = df_responses["Q3"].map(continents_dict)
country_long_name_dict = {"United States of America": "USA", 
                          "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                          "Iran, Islamic Republic of...": "Iran"
                         }
df_2018["Q3"] = df_2018["Q3"].replace(country_long_name_dict)
df_2019["Q3"] = df_2019["Q3"].replace(country_long_name_dict)
df_responses["Q3"] = df_responses["Q3"].replace(country_long_name_dict)

In [51]:
df_2018['year'] = 2018
df_2019['year'] = 2019
df_responses['year'] = 2020

total_responses = pd.concat([df_2018[['Q2','Q3','year']],
                             df_2019[['Q2','Q3','year']],
                             df_responses[['Q2','Q3','year']]])

total_responses=total_responses.groupby(['year','Q3']).count()

In [52]:
total_responses = total_responses.reset_index()
total_responses.columns = ["year", "country", "count"]

In [63]:
total_responses.sort_values(by=["year","count"], inplace=True, ascending=[True, True])

In [64]:
tl = Timeline()
for i in range(2018, 2021):
    data = total_responses.loc[total_responses.year == i].tail(20)
    bar=Bar().add_xaxis(data['country'].values.tolist())\
    .add_yaxis('num responses', data['count'].values.tolist())\
    .set_series_opts(label_opts=opts.LabelOpts(position="inside"))\
    .reversal_axis()\
    .set_global_opts(title_opts=opts.TitleOpts(title="Number of respondents wrt Country", subtitle="2018 - 2020 Kaggle Survey"),
                 yaxis_opts=opts.AxisOpts(name='Country'), xaxis_opts=opts.AxisOpts(name='Count'))
    
    tl.add(bar, "year:{}".format(i))
    
tl.add_schema(is_auto_play=True, play_interval=2000)
tl.render('fig5.html')

'/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/fig5.html'

In [54]:
asia_df = df_responses[df_responses["Continents"]=="Asia"]
america_df = df_responses[df_responses["Continents"]=="America"]
europe_df = df_responses[df_responses["Continents"]=="Europe"]
africa_df = df_responses[df_responses["Continents"]=="Africa"]
australia_df = df_responses[df_responses["Continents"]=="Australia"]
others_df = df_responses[df_responses["Continents"]=="Others"]

In [55]:
continents_count_df = pd.DataFrame(df_responses["Q2"].value_counts())
continents_count_df = continents_count_df.reset_index(drop=False)
continents_count_df.columns = ["Gender","Count"]

asia_country_count_df = pd.DataFrame(asia_df["Q2"].value_counts())
asia_country_count_df = asia_country_count_df.reset_index(drop=False)
asia_country_count_df.columns = ["Gender","Count"]

america_country_count_df = pd.DataFrame(america_df["Q2"].value_counts())
america_country_count_df = america_country_count_df.reset_index(drop=False)
america_country_count_df.columns = ["Gender","Count"]

europe_country_count_df = pd.DataFrame(europe_df["Q2"].value_counts())
europe_country_count_df = europe_country_count_df.reset_index(drop=False)
europe_country_count_df.columns = ["Gender","Count"]

others_country_count_df = pd.DataFrame(others_df["Q2"].value_counts())
others_country_count_df = others_country_count_df.reset_index(drop=False)
others_country_count_df.columns = ["Gender","Count"]

africa_country_count_df = pd.DataFrame(africa_df["Q2"].value_counts())
africa_country_count_df = africa_country_count_df.reset_index(drop=False)
africa_country_count_df.columns = ["Gender","Count"]

australia_country_count_df = pd.DataFrame(australia_df["Q2"].value_counts())
australia_country_count_df = australia_country_count_df.reset_index(drop=False)
australia_country_count_df.columns = ["Gender","Count"]

In [56]:
pie_world = (
    Pie()
    .add(
        "World",
        [list(z) for z in zip(continents_count_df.Gender, continents_count_df.Count)],
        center=["10%", "50%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="World", position="center"),
    )
    .add(
        "Asia",
        [list(z) for z in zip(asia_country_count_df.Gender, asia_country_count_df.Count)],
        center=["30%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Asia", position="center"),
    )
    .add(
        "Europe",
        [list(z) for z in zip(europe_country_count_df.Gender, europe_country_count_df.Count)],
        center=["50%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Europe", position="center"),
    )
    .add(
        "Africa",
        [list(z) for z in zip(africa_country_count_df.Gender, africa_country_count_df.Count)],
        center=["70%", "30%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Africa", position="center"),
    )
    .add(
        "America",
        [list(z) for z in zip(america_country_count_df.Gender, america_country_count_df.Count)],
        center=["30%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="America", position="center"),
    )
    .add(
        "Australia",
        [list(z) for z in zip(australia_country_count_df.Gender, australia_country_count_df.Count)],
        center=["50%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Australia", position="center"),
    )
    .add(
        "Others",
        [list(z) for z in zip(others_country_count_df.Gender, others_country_count_df.Count)],
        center=["70%", "70%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(formatter="Others", position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Gender Distribution",subtitle="wrt Continents" ),
        legend_opts=opts.LegendOpts(
            type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
        ),
    )
)

pie.render('fig4.html')

'/home/harisyam/PycharmProjects/ML_WS/2020_kaggle_survey/fig4.html'